In [29]:
from typing import List, Dict

# Cashu Mints

6 NUTS to implement. 

Things to do:

- [x] Generate pubkeys from a seed
    - different pubkeys for different amounts??
    - only one active key set
- [ ] Signing of tokens
- [ ] Verify tokens
- [x] Calculate keyset Ids
- [ ] make it so that wallet can request to mint
        - store hash with invoice for lookup later
- [ ] mint tokens
    - verify `BlindedMessage`s
    - create promises
- [ ] melt tokens - when a token is spent we need to invalidate it
- [ ] split tokens - when a request to split is made, invalidate previous tokens and calculate value of new one
    

## [Generating a Keyset](https://github.com/cashubtc/nuts/blob/main/02.md#mints-generating-a-keyset)

```python
for i in range(MAX_ORDER):
	k_i = HASH_SHA256(s + D + i)[:32]
 ```

`s` - seed

`k_i` - priv key

`i` - index of amount value

`MAX_ORDER` - max token values supported 
   - Typically, `MAX_ORDER = 64`

`D` - derivation path

Check this out: https://github.com/cashubtc/nutshell/blob/main/cashu/core/crypto/keys.py#L18C1-L18C1 

In [30]:
from hashlib import sha256
from coincurve import PrivateKey

def generate_private_keyset(seed, MAX_ORDER, derivation_path):
    to_hash = [(seed + derivation_path + str(i)).encode() for i in range(MAX_ORDER)]
    return {
        2 ** idx: PrivateKey(
            sha256(to_hash).digest()[:32]
        )
        for idx, to_hash in enumerate(to_hash)
    }

In [31]:
seed = "johny_apple"
MAX_ORDER = 8
derivation_path = "/0/0/0/0"
    
keys = generate_private_keyset(seed, MAX_ORDER, derivation_path)

keys


{1: <coincurve.keys.PrivateKey at 0x7f1698756d70>,
 2: <coincurve.keys.PrivateKey at 0x7f16986efcd0>,
 4: <coincurve.keys.PrivateKey at 0x7f1698728a60>,
 8: <coincurve.keys.PrivateKey at 0x7f1698667f70>,
 16: <coincurve.keys.PrivateKey at 0x7f1698650040>,
 32: <coincurve.keys.PrivateKey at 0x7f16986501f0>,
 64: <coincurve.keys.PrivateKey at 0x7f16986506a0>,
 128: <coincurve.keys.PrivateKey at 0x7f1698652230>}

## [Public Keyset](https://github.com/cashubtc/nuts/blob/main/02.md#nut-02-keysets-and-keyset-id)
A keyset is a set of public keys that the mint Bob generates and shares with its users. The mint MUST use the compressed public key format. The keyset refers to the set of public keys that each correspond to the amount values that the mint supports (e.g. 1, 2, 4, 8, ...) respectively.

In [32]:
from coincurve import PublicKey

def get_keyset(priv_keyset):
    pub_keyset = {}
    for idx, pk in priv_keyset.items():
        from coincurve import PublicKey
        pub_keyset[idx] = pk.public_key
    return pub_keyset

In [33]:
public_keyset = get_keyset(keys)
public_keyset

{1: <coincurve.keys.PublicKey at 0x7f169872b430>,
 2: <coincurve.keys.PublicKey at 0x7f16a1dfeb30>,
 4: <coincurve.keys.PublicKey at 0x7f1698755300>,
 8: <coincurve.keys.PublicKey at 0x7f1698667be0>,
 16: <coincurve.keys.PublicKey at 0x7f1698651ed0>,
 32: <coincurve.keys.PublicKey at 0x7f16986524d0>,
 64: <coincurve.keys.PublicKey at 0x7f1698652650>,
 128: <coincurve.keys.PublicKey at 0x7f16a1c985b0>}

## [Deriving Keyset ID](https://github.com/cashubtc/nuts/blob/main/02.md#keyset-id)

1.  sort keyset by amount
2.  concatenate all (sorted) public keys to one string
3.  HASH_SHA256 the concatenated public keys
4.  take the first 12 characters of the hash

In [34]:
from base64 import b64encode
from coincurve import PublicKey

def derive_keyset_id(keys: Dict[int, PublicKey]) -> str:
    sorted_keys = dict(sorted(keys.items()))
    pubkeys_concat = b"".join([p.format() for p in sorted_keys.values()])
    return "00" + sha256(pubkeys_concat).hexdigest()[:14]


def urlsafe_to_keyset_id(keyset_id_url_safe):
    return keyset_id_url_safe.replace('_', '/').replace('+', '-')

def keyset_id_to_urlsafe(keyset_id):
    return keyset_id.replace('/', '_').replace('-', '+')
    

In [35]:
keyset_id = derive_keyset_id(public_keyset)
print("KeysetID: ", keyset_id)

url_safe_keyset_id = keyset_id_to_urlsafe(keyset_id)
print ("URL Safe: ", url_safe_keyset_id)

assert keyset_id == urlsafe_to_keyset_id(url_safe_keyset_id)

KeysetID:  00762723599b57f8
URL Safe:  00762723599b57f8


## Questions

- What do we use as the derivation path `D` when generating a keyset? Can it be arbitrary?

- do we store each priv key? Or just the derivation path and keyset ID then generate keys each time for signing? This way we only store the seed?
- Does the mint have one single pubkey and single prive other than masterkey??
  - https://github.com/cashubtc/nutshell/blob/main/cashu/core/crypto/keys.py#L35
  - https://github.com/cashubtc/nutshell/blob/main/cashu/mint/ledger.py#L58

# [Blind Diffie-Hellmann key exchange (BDHKE)](https://github.com/cashubtc/nuts/blob/main/00.md#blind-diffie-hellmann-key-exchange-bdhke) 

## Bob (mint)

- `k` private key of mint (one for each amount)
- `K` public key of mint
- `Q` promise (blinded signature)

## Alice (user)

- `x` random string (secret message), corresponds to point `Y` on curve
- `r` private key (blinding factor)
- `T` blinded message
- `Z` proof (unblinded signature)

 ---

- Mint `Bob` publishes public key `K = kG` 
- `Alice` picks secret `x` and computes `Y = hash_to_curve(x)`
- `Alice` sends to `Bob`: `B_ = Y + rG` with `r` being a random blinding factor (**blinding**)
- `Bob` sends back to `Alice` blinded key: `C_ = kB_` (these two steps are the DH key exchange) (**signing**)
- `Alice` can calculate the unblinded key as `C_ - rK = kY + krG - krG = kY = C` (**unblinding**)
- Alice can take the pair `(x, C)` as a token and can send it to `Carol`.
- `Carol` can send `(x, C)` to `Bob` who then checks that `k*hash_to_curve(x) == C` (**verification**), and if so treats it as a valid spend of a token, adding `x`  to the list of spent secrets.

**Who knows what??**
Carol: K, x, Y, B_, r, C_

Bob: K, k, B_, C_

**Who doesn't know what??**

Carol: k

Bob: x, r, Y

In [36]:
lightning_bin_dir = "/usr/local/bin"
lightning_dir = "/tmp/l1"
l1cli = f"{lightning_bin_dir}/lightning-cli --lightning-dir={lightning_dir}"



In [37]:
pubkeys = !"$lightning_bin_dir"/lightning-cli --lightning-dir="$lightning_dir" cashu-get-keys | jq -r .keysets[0].keys

In [38]:
pubkeys

['{',
 '  "1": "02b53a33e8b70644d83e5700b2dfa45b7fd7bb1ed597a6f4f2408e79d2ed316cef",',
 '  "2": "034c307cfa8736158e2bb0435153f805e7d6d6ec95249eee32f31effe4fef87e24",',
 '  "4": "02837b0c27e0e1c00934925031e10c41e7a2b926eb1b6e8d5846024b64b16964cc",',
 '  "8": "0232eaf6f41725360b28ba5242243616099dce2f17aaf5fa7241bb4015d6461495",',
 '  "16": "031b06e936ace0e0e449a5eeabcb0dc39a11b72650142cc653d008dce482004178",',
 '  "32": "03053d1a6e05e1e12537129dad7c983cc1845279f3051df7d37956cdb09d633f29",',
 '  "64": "02f81334344f31fe0b919bd9fa40e4a5b0b4e4d4dbd017cdf6dbe66d1fbfae1975",',
 '  "128": "02ee1a9c0b7018150ff71ce3f2207011f034a6a5a81d40f2066105254993b4b305"',
 '}']

In [39]:
privkeys =  !"$lightning_bin_dir"/lightning-cli --lightning-dir="$lightning_dir" cashu-dev-get-privkeys
privkeys

['{',
 '   "1": "78196135e2121c6750e0d3ed7669cc43a83107d79dbfbb253c069292ec83c59a",',
 '   "2": "62d469a41ab8e82cb95dce66a8562f8a241d9f8c1c9dbaac41829d46d1856bb7",',
 '   "4": "3375381bb41a9cb1b1a47d78d37e4a703332af50ef4d1661a8892ba3f0e42f8c",',
 '   "8": "5c47dc806dac67518dd8355dc1c2b29daf73a88195913b5b1b8e5bf319d40ae7",',
 '   "16": "2f8cae4671fadfec38df2d8afa2b57b5303c5035488e23d373b2bb96139b9992",',
 '   "32": "226d7f6783a73927ece9009124589aaf107ea0edafb9f4c7c5358301b54e7888",',
 '   "64": "52e4b9328d26a927e0473e806f52fa058027c649f9bfee45f8ed13c585892fcd",',
 '   "128": "2207ce69cd468de1790ad1c8a5c9516ac6f6068067bf00b1e4545e134bb70fb8"',
 '}']

In [40]:
import json
json_string = ''.join(privkeys)
    
# Parse the JSON string
privkeys_dict = json.loads(json_string)

# Now privkeys_dict is a Python dictionary
# Access the private key for key "1"
privkey_1 = privkeys_dict.get("1", "default_value")
privkey_2 = privkeys_dict.get("2", "default_value")
privkey_4 = privkeys_dict.get("4", "default_value")
privkey_1

'78196135e2121c6750e0d3ed7669cc43a83107d79dbfbb253c069292ec83c59a'

In [41]:
pub_keys_json_string = ''.join(pubkeys)
pubkey_dict = json.loads(pub_keys_json_string)
pubkey_1 = pubkey_dict.get("1", "default_value")
pubkey_2 = pubkey_dict.get("2", "default_value")
pubkey_4 = pubkey_dict.get("4", "default_value")
pubkey_1

'02b53a33e8b70644d83e5700b2dfa45b7fd7bb1ed597a6f4f2408e79d2ed316cef'

In [42]:
amount = 1
k = PrivateKey().from_hex(privkey_1)
K = k.public_key
assert K == PublicKey(bytes.fromhex(pubkey_1))

In [43]:
def hash_to_curve(x_bytes):
    # Hash the secret using SHA-256
    hash_value = sha256(x_bytes).digest()
    # Create a public key Y from the hashed secret
    Y = PublicKey.from_secret(hash_value)
    return Y

In [44]:
x_secret = b'a secret'

In [45]:
Y = hash_to_curve(x_secret)
Y

**Alice sends to Bob:** `B_ = Y + rG`

In [46]:
from coincurve import PrivateKey
r = PrivateKey()
R = r.public_key
B_ = PublicKey.combine_keys([Y, R]).format().hex()
B_

'027f50d1a8cce147ab6e7dbaf2d39e2e2f690c2748eebf498902aa131412ee638e'

**Bob send to Alice:** `C_ = kB_`

In [47]:
# BlindedMessage: https://github.com/cashubtc/nuts/blob/main/00.md#blindedmessage  

# @plugin.method("cashu-sign")# TODO: add id to specify which keyset to use
def sign(B_, k: PrivateKey): 
    # B_ = Y + rG with r being a random blinding factor (blinding)
    # C_ = kB_ (these two steps are the DH key exchange) (signing)
    B_bytes = bytes.fromhex(B_)
    C_ = PublicKey(B_bytes).multiply(k.secret)
    return C_

In [48]:
# priv key that corresponds to token value `1`
C_ = sign(B_, k)
C_.format().hex()

'02220b777a868c26543fccf30f2af625d6b4925adbc0c5ebc30d92fedafe995155'

**Alice:**   `C_ - rK = kY + krG - krG = kY = C`

In [49]:
# order of the finite field
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663

rK = PublicKey(K.format()).multiply(r.secret)

x,y = rK.point()
neg_rK = PublicKey.from_point(x, (p - y) % p)

# C = C_ - rK
C = PublicKey.combine_keys([C_, neg_rK])

# kY should equal C
kY = Y.multiply(k.secret)
kY == C

True

Now the token is `(x, c)`

In [22]:
token = (x_secret, C)
token

(b'a secret', <coincurve.keys.PublicKey at 0x7f16a0255bd0>)

Carol can now redeem with Bob.

Bob verifies the token with:

`k*hash_to_curve(x) == C`

In [23]:
x_secret, C = token
def verify_token(C, secret, k: PrivateKey):
    # k*hash_to_curve(x) == C
    # secret_bytes = secret.encode()
    Y = hash_to_curve(secret)
    kY = Y.multiply(k.secret)
    return kY.format().hex() == C

verify_token(C.format().hex(), x_secret, k)

True

If valid spend the **add `x_secret` to list of spent secrets**

In [24]:
l1 = "/usr/local/bin/lightning-cli --lightning-dir=/tmp/l1"
l2 = "/usr/local/bin/lightning-cli --lightning-dir=/tmp/l2"

In [25]:
### Test Mint Flow

In [26]:
mint_amount = 7

#### Request quote for minting

In [27]:
mint_quote = !$l1 -k cashu-get-quote amount=$mint_amount unit="sat" | jq -r 

json_mint_quote = json.loads(''.join(mint_quote))
mint_quote_id = json_mint_quote["quote"]
print("quoteId:", mint_quote_id)
invoice_to_pay = json_mint_quote["request"]
print("bolt11:", invoice_to_pay)


KeyError: 'quote'

#### L2 pays invoice 

In [ ]:
# pay invoice with l2
! $l2 pay "$invoice_to_pay"

#### Check status of mint

In [ ]:
!$l1 cashu-check-quote "$mint_quote_id"

In [ ]:
# generate the blinded messages
from coincurve import PrivateKey, PublicKey
from hashlib import sha256
secrets = ["1", "2", "4"]
amounts = [1,2,4]
Ys = [hash_to_curve(s.encode()) for s in secrets]
rs = [PrivateKey() for _ in range(3)]
B_messages = []
for secret, amount, Y, r in zip(secrets, amounts, Ys, rs):
    R = r.public_key
    B_ = PublicKey.combine_keys([Y, R]).format().hex()
    B_messages.append({"amount": amount, "B_": B_})

json.dumps(B_messages)
# r = PrivateKey()
# R = r.public_key
# B_ = PublicKey.combine_keys([Y, R]).format().hex()
# B_

In [ ]:
blinded_sigs = ! /usr/local/bin/lightning-cli --lightning-dir=/tmp/l1 cashu-mint 1373345158dd1510 '[{"amount": 1, "B_": "021a40cba4ec6ced1743350897e4c7ad46676624577e14fb1a0939819bb48f0f5b"}, {"amount": 2, "B_": "0223bb3a32c1cca89ca19c71bbcd1b40bb7f85696d494fc1894afe7805bd1abea9"}, {"amount": 4, "B_": "0358edb2a9b6c6b3e573070a6ecb77a703a6a2793303a0e4009205e7f909646875"}]'
blinded_sigs =json.loads(''.join(blinded_sigs))
blinded_sigs

In [ ]:
# l2 needs an invoice to give th emint for melting
l2_invoice = ! /usr/local/bin/lightning-cli --lightning-dir=/tmp/l2 invoice 7000 $RANDOM description | jq -r  .'bolt11'
l2_invoice = l2_invoice[0]
l2_invoice

In [ ]:
request = l2_invoice
unit = "sat"

In [ ]:
# Getting a melt quote
melt_quote = !$l1_cli -k cashu-melt-quote req=$request unit="sat" | jq -r .'quote'
melt_quote = melt_quote[0]
melt_quote

In [ ]:
# Check melt quote status
!$l1_cli cashu-check-melt-quote $melt_quote

In [ ]:
# Pay invoice
!$l1_cli pay $l2_invoice

In [28]:
# construct inputs
# order of the finite field
# p = 115792089237316195423570985008687907853269984665640564039457584007908834671663

# rK = PublicKey(K.format()).multiply(r.secret)

# x,y = rK.point()
# neg_rK = PublicKey.from_point(x, (p - y) % p)

# # C = C_ - rK
# C = PublicKey.combine_keys([C_, neg_rK])

# # kY should equal C
# kY = Y.multiply(k.secret)
# kY == C

def subtract_points(pt1: PublicKey, pt2:  PublicKey):
    p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
    x2, y2 = pt2.point()
    neg_pt2 = PublicKey.from_point(x2, (p - y2) % p)
    difference = PublicKey.combine_keys([C_, neg_pt2])
    return difference

inputs = []
for output, r, secret in zip(blinded_sigs, rs, secrets):
    K = pubkey_dict.get(str(output["amount"]), "default_value")
    K_bytes = bytes.fromhex(K)
    rK = PublicKey(K_bytes).multiply(r.secret)
    C_ = PublicKey(bytes.fromhex(output["C_"]))
    C = subtract_points(C_, rK)
    inputs.append({
        "amount": output["amount"],
        "C": C.format().hex(),
        "id": output["id"],
        "secret":secret
    })

json.dumps(inputs)

    

NameError: name 'blinded_sigs' is not defined

In [ ]:
!/usr/local/bin/lightning-cli --lightning-dir=/tmp/l1 -k cashu-melt quote=ab34680b4acab67b inputs='[{"amount": 1, "C": "02dad6625e6ca42caa8197f801ded28e10f0b7a3cf81a8f525984c825774b1ee64", "id": "00b2f181c83b11aa", "secret": "1"}, {"amount": 2, "C": "03eed603517a7cba2accb07834990d6c74e0ac6c7757726740a05e9ea538625f0f", "id": "00b2f181c83b11aa", "secret": "2"}, {"amount": 4, "C": "02a6a27f1394eb7000b923fe56e45da8d0fa15f5d6edf1be781438416c9c13e9b2", "id": "00b2f181c83b11aa", "secret": "4"}]'

In [ ]:
def verify_token(C, secret, k: PrivateKey):
    # k*hash_to_curve(x) == C
    secret_bytes = secret.encode()
    Y = hash_to_curve(secret_bytes)
    kY = Y.multiply(k.secret)
    return kY.format().hex() == C

verify_token(

### Test `cashu-sign` Method

In [ ]:
!/nix/store/0bq00qd0scchy6vasjpqxqg53aijcca3-clightning-v23.11rc1/bin/lightning-cli --lightning-dir=/home/gudnuf/pyshu_mint/.lightning_nodes/l1 -k cashu-sign amount=3 B_="034110bf4bff1838c9f79a5de86c6215b9cf0fef081f3337e40f37a016691fc302" | jq -r

# [Requesting a Mint](https://github.com/cashubtc/nuts/blob/main/03.md)
1. Wallet sends `GET /mint?amount=<amount_sat>`
2. Mint responds with invoice and hash:

```json
{
  "pr": "lnbc100n1p3kdrv5sp5lpdxzghe5j67q...",
  "hash": "67d1d9ea6ada225c115418671b64a..."
}
```
3. Wallet pays invoice
4. Wallet requests tokens: `POST /mint?hash=<hash>` and `BlindedMessages` in request body
5.  d

#### `BlindedMessage` (outputs)
```json
{
  "amount": int,
  "B_": hex_str
}
```

`BlindedMessages` is an array of `BlindedMessage`s

**hash must be unique and stored with amount_sat**


##### Verify `BlindedMessages`
- `amount`s must add to a maximum of `<amount_sat>`
   - find amouunt sat with the `hash`
- invoice must be paid
- `BlindedMessages` are valid

#### `PostMintResponse`

If request for token is valid, then mint responds with `BlindedSignature`s (`C_`)

```json
{
"promises":
  [
    {
    "id": "DSAl9nvvyfva",
    "amount": 2,
    "C_": "03e61daa438fc7bcc53f6920ec6c8c357c24094fb04c1fc60e2606df4910b21ffb"
    },
    {
    "id": "DSAl9nvvyfva",
    "amount": 8,
    "C_": "03fd4ce5a16b65576145949e6f99f445f8249fee17c606b688b504a849cdc452de"
    },
  ]
}
```


#### Unblinfing Signatures

Once the wallet receives a response from mint, then unblind to create `Proofs`.

##### `Proofs`
```json
{
"proofs" : 
  [
    {
    "id": "DSAl9nvvyfva", //keyset id
    "amount": 2, //token amount value
    "secret": "S+tDfc1Lfsrb06zaRdVTed6Izg", // secret used by wallet in the signing process
    "C": "0242b0fb43804d8ba9a64ceef249ad7a60f42c15fe6d4907238b05e857527832a3" // unblinded sig
    },
    {
    ...
    }
  ]
}
```

## Questions
- What does the mint need to store? Just the secrets that have been spent?
    - If the mint signs a token where that secret is hidden, then the mint know that they issued the token, but there is one secret for one token, so if secret is spent thats all we need to know... right?? right??
    - real question: one secret for one token??

# [Melting Tokens](https://github.com/cashubtc/nuts/blob/main/05.md#nut-05-melting-tokens)
Opposite of minting

1. wallet gets maximum fees `POST /fees` with ln invoice
2. mint responds with fees
3. wallet send `Proofs` along with a lightning invoice `POST /melt` (mint expexts `Proofs` of at least `total_amount = amount + fee_reserve`
4. mint responds with:

```json
{
"paid": true,
"preimage": "da225c115418671b64a67d1d9ea6a..."
}
```

Once paid, **mint should mark all secrets from tokens as spent**

# [Splitting Tokens](https://github.com/cashubtc/nuts/blob/main/06.md#nut-06-split-tokens)

- Involves `Proofs` and `BlindedMessages` in the request to split
- Mint verifies, then invalidates `Proofs` --> then issues new promises `BlindedSignatures`


- Wallet requests split `POST /split` with `Proofs` and `BlindedMessages` in request body
- Mint responds with `BlindedSignatures` if all good

Nice function for getting split amounts https://github.com/cashubtc/nutshell/blob/main/cashu/core/split.py

# What is needed to run a mint??

- ability to blind sign 
- ability to verify that I signed 
- database to keep track of things so we don't lose money and be sad
  - spent secrets
  - `<hash>:<amount_sat>` and maybe an invoice for the minting process??
- API for interacting with wallets